v0: DBPIA, Scienceon Rule계산(Yop + co_authorship + keyword)
v1: title 조건 추가, jaro_winkler_metric 추가
v2: site 조건 변경
v3: Rule계산(joc + yop + title_cossim + abstract_cossim) 추가, TF-IDF 추가, 교신저자 비교
v4: co_authorship 비율추가
v5: site 조건 수정, 함수/변수이름 변경, filter 함수 추가, title_cossim + abstract_cossim 삭제, check_college 추가

In [30]:
from bson.objectid import ObjectId
from pymongo import MongoClient
import numpy as np
import jaro
import re

def isEnglishOrKorean(input_s):
    k_count = 0
    e_count = 0
    for c in input_s:
        if ord('가') <= ord(c) <= ord('힣'):
            k_count+=1
        elif ord('a') <= ord(c.lower()) <= ord('z'):
            e_count+=1
    return "k" if k_count>1 else "e"

def check_college(univ0):
    branch_set = ['성균관대학교', '건국대학교', '한양대학교']
    univName = client['PUBLIC']['CollegeName']
    univ1 = re.sub("산학협력단|병원","",univ0)
    univ2 = re.sub("대학교","대학교 ",univ1)

    try:
        if isEnglishOrKorean(univ0) == 'e':
            univ0 = univ0.upper()
            univ0 = univ0.replace('.', ',')
            univ = univ0.split(', ')
        else:
            univ = univ2.replace(",", "").split()
            univ = list(set(univ))   
            
        for uni in univ:
            if uni in branch_set:
                if ("ERICA" or "에리카") in univ0:
                    univ[univ.index("한양대학교")] = "한양대학교(ERICA캠퍼스)"
                elif ("글로컬" or "GLOCAL") in univ0:
                    univ[univ.index("건국대학교")] = "건국대학교 GLOCAL(글로컬)캠퍼스"
                elif "자연과학캠퍼스" in univ0:
                    univ[univ.index("성균관대학교")] = "성균관대학교(자연과학캠퍼스)"

        univs = '{"$or": ['
        for u in range(len(univ)):
            if univ[-1] == univ[u]:
                univs += '{"inputName": "' + univ[u] + '"}'
            else:
                univs += '{"inputName": "' + univ[u] + '"}, '
        univs += ']}'

        univ_query = univName.find_one(eval(univs))

        if univ_query is None:
            print("Search inst None")
            return univ0, False
        else:
            #print("rawInput:[",univ0,"]","queryOutput:" ,univ_query['originalName'])
            return univ_query['originalName'], True #univ0, univ_query
        
    except SyntaxError as e:
        print(e)
        print(univ0)
        return univ0, False

def filter(site, rawdata):
    instbool = False
    if site == 'NTIS' :
        inst, instbool = check_college('성균관대학교(자연과학캠퍼스)')
        inst = rawdata['ldAgency']

        coauthor = rawdata['rsc'].split(";")
        year = int(rawdata['prdStart'][:4])
        keyword = rawdata['koKeyword'].split(",")
        journal = ""
        conference = ""
        title = ""
        mng = rawdata['mng']

    else :
        inst, instbool = check_college('성균관대학교')
        #inst = rawdata['author_inst'].split(";")[rawdata['author'].split(";").index(name)]
        coauthor = rawdata['author'].split(";")[1:-1]
        year = int(rawdata['issue_year'][:4])
        keyword = rawdata['paper_keyword'].replace(" ", "").split(".")
        journal = rawdata['journal']
        conference = rawdata['issue_inst']
        title = rawdata['title']
        mng = rawdata['author'].split(";")[-1]
        #abstract = rawdata['abstract']

    return inst, coauthor, year, keyword, journal, conference, title, instbool
    #return inst, coauthor, year, keyword, journal, conference, title, mng, abstract

def Secondary_filter(name, site1, raw_one1, site2, raw_one2):
    inst1, coauthor1, year1, keyword1, journal1, conference1, title1, instbool1 = filter(site1, raw_one1)
    inst2, coauthor2, year2, keyword2, journal2, conference2, title2, instbool2 = filter(site2, raw_one2)
    print(inst1, inst2)
    inst = 0

    if instbool1 and instbool2:
        if inst1 == inst2:
            inst = 1
    else:
        inst = jaro.jaro_winkler_metric(inst1, inst2)
    weight = 0
    joc = 0
    
    if name in coauthor1:
        coauthor1.remove(name)

    if name in coauthor2:
        coauthor2.remove(name)

    co_author_count = len([i for i in coauthor1 if i in coauthor2])

    if site1 != 'NTIS' and site2 != 'NTIS' :
        if title1 == title2 : # or inst >= 0.8:
            weight = 4
            return weight

        else:
            joc = 1 if journal1 == journal2 and conference1 == conference2 else -1
    else:
        if inst >= 0.8:
            weight = 3
            return weight
            
    yop = -(2*(abs(year1-year2)/10)-1)
            
    if len(coauthor1) > len(coauthor2):
        co_author_ratio = co_author_count/len(coauthor1)
    else:
        co_author_ratio = co_author_count/len(coauthor2)
                
    co_authorship = (1 - np.exp(-co_author_count))/2 + (co_author_ratio/2)
    keyword = 1 - np.exp(-len([i for i in keyword1 if i in keyword2]))

    print(f'joc: {joc} | yop: {yop} | co_authorship: {co_authorship} | keyword: {keyword}')

    weight = joc + yop + co_authorship + keyword

    return weight

In [35]:
client = MongoClient('mongodb://203.255.92.141:27017', authSource='admin')

name = '이상원'
site1 = 'NTIS'
site2 = 'NTIS'
input_paper1 = ObjectId('61cbfaf43b642afa40456740')
input_paper2 = ObjectId('61cbfaae3b642afa40456002')
'''
NTIS
ObjectId('61cbfad13b642afa4045638d') 성균관대학교(자연과학캠퍼스)
제조용 로봇/공정설비 다운타임 최소화를 통한 자동차 부품 모듈 생산성 및 수율 향상(30%)을 위한 인공지능 활용 고장예방 및 공정품질 개선 원천 기술 개발
한순영;허지웅;정상익;임수연;오기용;최해진;장성우;PHAM VAN HUAN;강병주;신동준;이지웅;김정섭;노인웅;이창수;이신영;이승열
209146726136
2018
ObjectId('61cbfac53b642afa4045630f') 성균관대학교(자연과학캠퍼스)
제조용 로봇/공정설비 다운타임 최소화를 통한 자동차 부품 모듈 생산성 및 수율 향상(30%)을 위한 인공지능 활용 고장예방 및 공정품질 개선 원천 기술 개발
한승재;조승연;PHAM VAN HUAN;이용호;NGUYEN DUY HUNG;이현석;김민건;선호서;최해진;김정섭;이지민;이기훈;김준호;정상익;노인웅;한순영;장성우;이지웅;이창수;이신영;임수연;임형남;이승열
209146726136
2019
ObjectId('61cbfaae3b642afa40456002') 성균관대학교(자연과학캠퍼스)
제조용 로봇/공정설비 다운타임 최소화를 통한 자동차 부품 모듈 생산성 및 수율 향상(30%)을 위한 인공지능 활용 고장예방 및 공정품질 개선 원천 기술 개발
이용호;김근영;최해진;조승연;노인웅;최영운;전재훈;이지민;이창호;한순영;이기훈;전용재;한승재;조준형;신혜원;PHAM VAN HUAN;이지웅;임수연;임형남;김진우
209146726136
2020
ObjectId('61cbfa9a3b642afa40455e98') 성균관대학교(자연과학캠퍼스)
제조용 로봇/공정설비 다운타임 최소화를 통한 자동차 부품 모듈 생산성 및 수율 향상(30%)을 위한 인공지능 활용 고장예방 및 공정품질 개선 원천 기술 개발
조승연;노인웅;이용호;한승재;구본상;이지환;이지웅;임수연;최해진;박소영;김민건;신준희;김수종;강병권;전용재;김진우
209146726136
2021
ObjectId('61cbfaf43b642afa40456740') 성균관대학교(자연과학캠퍼스)
3D 프린팅 공정의 스마트화를 위한 사물인터넷 및 사이버 물리 시스템 연계 스마트 팩토리 기술 개발
None
209146726136
2016
ObjectId('61cbfae33b642afa40456558') 성균관대학교(자연과학캠퍼스)
3D 프린팅 공정의 스마트화를 위한 사물인터넷 및 사이버 물리 시스템 연계 스마트 팩토리 기술 개발
None
209146726136
2017

ObjectId('61cbfaf23b642afa40456700') 성균관대학교
사회연결망 이론을 활용한 사용자 지향적 사물인터넷 프레임워크 개발
224907157190
None
2016
ObjectId('61cbfaf23b642afa40456700') 성균관대학교
사회연결망 이론을 활용한 사용자 지향적 사물인터넷 프레임워크 개발
224907157190
None
2016

SCIENCEON
ObjectId('61cbfa95597f83aefb455c3a') Department of Interaction Science, Sungkyunkwan University
엣지-클라우드 협업 기반 인터랙티브 사이니지 제공 플랫폼
s304141
문재원;금승우;이상원;
2019
KCI
ObjectId('61cbfb6bf72c82d20d455ca1') 성균관대학교
엣지-클라우드 협업 기반 인터랙티브 사이니지 제공 플랫폼
s821
문재원;금승우;이상원;
2019
'''
if site1 == 'Scienceon' :
    site_raw1 = client['SCIENCEON']['Rawdata']
elif site1 == 'NTIS' :
    site_raw1 = client['NTIS']['Rawdata']
elif site1 == 'DBPIA' :
    site_raw1 = client['DBPIA']['Rawdata']

if site2 == 'Scienceon' :
    site_raw2 = client['SCIENCEON']['Rawdata']
elif site2 == 'NTIS' :
    site_raw2 = client['NTIS']['Rawdata']
elif site2 == 'DBPIA' :
    site_raw2 = client['DBPIA']['Rawdata']

input_raw1 = site_raw1.find_one({ '_id' : input_paper1 })
input_raw2 = site_raw2.find_one({ '_id' : input_paper2 })

Secondary_filter(name, site1, input_raw1, site2, input_raw2)

성균관대학교(자연과학캠퍼스) 성균관대학교산학협력단
joc: 0 | yop: 0.19999999999999996 | co_authorship: 0.0 | keyword: 0.0


0.19999999999999996